In [1]:
%time
import numpy as np
import pandas as pd
import math
import sys
import os
import string
import contractions
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import SVG
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from pylab import *
from textblob import TextBlob


import nltk
from nltk import punkt
from nltk import wordnet
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize

import sklearn
import keras
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import spacy
from spacy import displacy

import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel,CoherenceModel,LsiModel,HdpModel
#from gensim.models.wrappers import LdaMallet
import pyLDAvis.gensim
import tomotopy as tp
import pyLDAvis
import bertopic
from sklearn.decomposition import NMF, LatentDirichletAllocation
import torch

from bertopic import BERTopic

import warnings
warnings.filterwarnings("ignore")

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

stopword=set(STOPWORDS)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [2]:
train1=pd.read_csv("jigsaw-toxic-comment-train.csv")
train2=pd.read_csv("jigsaw-unintended-bias-train.csv")
train2.toxic=train2.toxic.round().astype(int)

# 合并train1和train2
train_cat=pd.concat([
    train1[['comment_text','toxic']],
    train2[['comment_text','toxic']]
])

train_data=train_cat
train_data=train_data[train_data['toxic']==1]

In [3]:
# 大小写
train_data['text_clean']=train_data['comment_text'].apply(lambda x: x.lower())

# 缩写
train_data['text_clean']=train_data['text_clean'].apply(lambda x: contractions.fix(x))

# 删除噪声
## URLs
def remove_URL(text):
    return re.sub(r"https?://\S+|www\.\S+", "", text)

train_data['text_clean']=train_data['text_clean'].apply(lambda x: remove_URL(x))

## HTML
def remove_html(text):
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)

train_data['text_clean']=train_data['text_clean'].apply(lambda x: remove_html(x))

## Non-ASCI
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7f]',r'', text)

train_data['text_clean']=train_data['text_clean'].apply(lambda x: remove_non_ascii(x))

## 特殊符号（符号、表情符号、其他图形字符）
def remove_special_characters(text):
    emoji_pattern=re.compile(
    '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'',text)

train_data['text_clean']=train_data['text_clean'].apply(lambda x: remove_special_characters(x))
display(train_data.head())

# 删除标点符号
def remove_punct(text):
    return text.translate(str.maketrans('', '',string.punctuation))

train_data['text_clean']=train_data['text_clean'].apply(lambda x: remove_punct(x))

train_data.head()

,comment_text,toxic,text_clean
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey... what is it..\n@ | talk .\nwhat is it......
16,"Bye! \n\nDon't look, come or think of comming ...",1,"bye! \n\ndo not look, come or think of comming..."
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you are gay or antisemmitian? \n\narchangel wh...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,"fuck your filthy mother in the ass, dry!"


,comment_text,toxic,text_clean
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what is it\n talk \nwhat is it an exclusi...
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye \n\ndo not look come or think of comming b...
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you are gay or antisemmitian \n\narchangel whi...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,fuck your filthy mother in the ass dry


In [4]:
sys.setrecursionlimit(100000)
# 分词
train_data['tokenized']=train_data['text_clean'].apply(word_tokenize)

# 删除停止词
train_data['stopwords_removed']=train_data['tokenized'].apply(lambda x: [word for word in x if word not in STOPWORDS])

# 词干提取
def porter_stemmer(text):
    stemmer=nltk.PorterStemmer()
    stems=[stemmer.stem(i) for i in text]
    return stems

train_data['porter_stemmer']=train_data['stopwords_removed'].apply(lambda x: porter_stemmer(x))

train_data.head()

,comment_text,toxic,text_clean,tokenized,stopwords_removed,porter_stemmer
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work,"[cocksucker, before, you, piss, around, on, my...","[cocksucker, piss, around, work]","[cocksuck, piss, around, work]"
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what is it\n talk \nwhat is it an exclusi...,"[hey, what, is, it, talk, what, is, it, an, ex...","[hey, talk, exclusive, group, wp, talibanswho,...","[hey, talk, exclus, group, wp, talibanswho, go..."
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye \n\ndo not look come or think of comming b...,"[bye, do, not, look, come, or, think, of, comm...","[bye, look, come, think, comming, back, tosser]","[bye, look, come, think, com, back, tosser]"
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you are gay or antisemmitian \n\narchangel whi...,"[you, are, gay, or, antisemmitian, archangel, ...","[gay, antisemmitian, archangel, white, tiger, ...","[gay, antisemmitian, archangel, white, tiger, ..."
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,fuck your filthy mother in the ass dry,"[fuck, your, filthy, mother, in, the, ass, dry]","[fuck, filthy, mother, ass, dry]","[fuck, filthi, mother, ass, dri]"


In [5]:
# 词性标注
wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    pos_tagged_text = t2.tag(text)
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

train_data['combined_postag_wnet'] = train_data['stopwords_removed'].apply(lambda x: pos_tag_wordnet(x))

def lemmatize_word(text):
    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma

lemmatizer=WordNetLemmatizer()

train_data['lemmatize_word_w_pos']=train_data['combined_postag_wnet'].apply(lambda x: lemmatize_word(x))
train_data['lemmatize_word_w_pos']=train_data['lemmatize_word_w_pos'].apply(lambda x: [word for word in x if word not in STOPWORDS])
train_data['lemmatize_text']=[' '.join(map(str,l)) for l in train_data['lemmatize_word_w_pos']]

train_data.head()

,comment_text,toxic,text_clean,tokenized,stopwords_removed,porter_stemmer,combined_postag_wnet,lemmatize_word_w_pos,lemmatize_text
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work,"[cocksucker, before, you, piss, around, on, my...","[cocksucker, piss, around, work]","[cocksuck, piss, around, work]","[(cocksucker, n), (piss, n), (around, r), (wor...","[cocksucker, piss, around, work]",cocksucker piss around work
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what is it\n talk \nwhat is it an exclusi...,"[hey, what, is, it, talk, what, is, it, an, ex...","[hey, talk, exclusive, group, wp, talibanswho,...","[hey, talk, exclus, group, wp, talibanswho, go...","[(hey, n), (talk, n), (exclusive, a), (group, ...","[hey, talk, exclusive, group, wp, talibanswho,...",hey talk exclusive group wp talibanswho good d...
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye \n\ndo not look come or think of comming b...,"[bye, do, not, look, come, or, think, of, comm...","[bye, look, come, think, comming, back, tosser]","[bye, look, come, think, com, back, tosser]","[(bye, n), (look, n), (come, v), (think, v), (...","[bye, look, come, think, comming, back, tosser]",bye look come think comming back tosser
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you are gay or antisemmitian \n\narchangel whi...,"[you, are, gay, or, antisemmitian, archangel, ...","[gay, antisemmitian, archangel, white, tiger, ...","[gay, antisemmitian, archangel, white, tiger, ...","[(gay, a), (antisemmitian, n), (archangel, n),...","[gay, antisemmitian, archangel, white, tiger, ...",gay antisemmitian archangel white tiger meow g...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,fuck your filthy mother in the ass dry,"[fuck, your, filthy, mother, in, the, ass, dry]","[fuck, filthy, mother, ass, dry]","[fuck, filthi, mother, ass, dri]","[(fuck, n), (filthy, n), (mother, n), (ass, n)...","[fuck, filthy, mother, dry]",fuck filthy mother dry


In [6]:
bigram = gensim.models.phrases.Phrases(train_data['lemmatize_word_w_pos'])
texts = [bigram[line] for line in train_data['lemmatize_word_w_pos']]
print(texts[0])

['cocksucker', 'piss', 'around', 'work']


In [7]:
dictionary=Dictionary(texts)
dictionary.filter_extremes(no_below=1,no_above=0.5,keep_n=500)
corpus=[dictionary.doc2bow(text) for text in texts]
print(corpus[1])

[(2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)]


In [8]:
# LSI
lsi_model=LsiModel(corpus=corpus,num_topics=4,id2word=dictionary)
lsi_model.show_topics(num_topics=5)

[(0,
  '0.718*"fuck" + 0.688*"go" + 0.070*"wikipedia" + 0.032*"kill" + 0.028*"murder" + 0.023*"will" + 0.013*"people" + 0.012*"say" + 0.011*"one" + 0.011*"shit"'),
 (1,
  '0.772*"will" + 0.188*"stupid" + -0.182*"fuck" + 0.178*"people" + 0.167*"trump" + 0.127*"go" + 0.116*"one" + 0.110*"say" + 0.103*"take" + 0.100*"know"'),
 (2,
  '0.573*"wikipedia" + -0.559*"go" + 0.480*"fuck" + 0.250*"will" + -0.147*"kill" + -0.146*"murder" + 0.106*"die" + 0.047*"fucking" + 0.043*"shit" + 0.039*"block"'),
 (3,
  '0.919*"stupid" + -0.318*"will" + 0.094*"people" + 0.069*"wikipedia" + 0.059*"say" + -0.059*"go" + 0.051*"one" + -0.049*"block" + 0.048*"trump" + 0.045*"think"')]

In [10]:
# HDP
hdp_model = HdpModel(corpus=corpus,id2word=dictionary)
hdp_model.show_topics()

[(0,
  '0.015*one + 0.013*people + 0.012*go + 0.011*will + 0.010*want + 0.010*trump + 0.010*think + 0.009*guy + 0.009*time + 0.008*know + 0.008*make + 0.007*real + 0.007*well + 0.007*say + 0.007*really + 0.006*now + 0.006*type + 0.006*start + 0.006*article + 0.006*excuse'),
 (1,
  '0.014*will + 0.012*go + 0.012*take + 0.012*need + 0.011*know + 0.011*say + 0.011*people + 0.010*day + 0.008*trump + 0.007*stupid + 0.007*see + 0.007*one + 0.007*perhaps + 0.007*every + 0.006*bad + 0.006*now + 0.006*think + 0.006*time + 0.006*want + 0.006*thing'),
 (2,
  '0.018*will + 0.011*people + 0.010*trump + 0.009*try + 0.009*say + 0.009*kill + 0.008*now + 0.008*probably + 0.008*go + 0.008*one + 0.008*think + 0.007*call + 0.007*hear + 0.007*man + 0.007*mean + 0.007*know + 0.007*see + 0.006*old + 0.006*child + 0.006*expect'),
 (3,
  '0.043*will + 0.009*trump + 0.008*live + 0.008*make + 0.008*people + 0.008*say + 0.008*kill + 0.008*go + 0.008*come + 0.007*need + 0.007*one + 0.007*stupid + 0.007*serve + 0.0

In [11]:
train_data.head()

,comment_text,toxic,text_clean,tokenized,stopwords_removed,porter_stemmer,combined_postag_wnet,lemmatize_word_w_pos,lemmatize_text
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,cocksucker before you piss around on my work,"[cocksucker, before, you, piss, around, on, my...","[cocksucker, piss, around, work]","[cocksuck, piss, around, work]","[(cocksucker, n), (piss, n), (around, r), (wor...","[cocksucker, piss, around, work]",cocksucker piss around work
12,Hey... what is it..\n@ | talk .\nWhat is it......,1,hey what is it\n talk \nwhat is it an exclusi...,"[hey, what, is, it, talk, what, is, it, an, ex...","[hey, talk, exclusive, group, wp, talibanswho,...","[hey, talk, exclus, group, wp, talibanswho, go...","[(hey, n), (talk, n), (exclusive, a), (group, ...","[hey, talk, exclusive, group, wp, talibanswho,...",hey talk exclusive group wp talibanswho good d...
16,"Bye! \n\nDon't look, come or think of comming ...",1,bye \n\ndo not look come or think of comming b...,"[bye, do, not, look, come, or, think, of, comm...","[bye, look, come, think, comming, back, tosser]","[bye, look, come, think, com, back, tosser]","[(bye, n), (look, n), (come, v), (think, v), (...","[bye, look, come, think, comming, back, tosser]",bye look come think comming back tosser
42,You are gay or antisemmitian? \n\nArchangel WH...,1,you are gay or antisemmitian \n\narchangel whi...,"[you, are, gay, or, antisemmitian, archangel, ...","[gay, antisemmitian, archangel, white, tiger, ...","[gay, antisemmitian, archangel, white, tiger, ...","[(gay, a), (antisemmitian, n), (archangel, n),...","[gay, antisemmitian, archangel, white, tiger, ...",gay antisemmitian archangel white tiger meow g...
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,fuck your filthy mother in the ass dry,"[fuck, your, filthy, mother, in, the, ass, dry]","[fuck, filthy, mother, ass, dry]","[fuck, filthi, mother, ass, dri]","[(fuck, n), (filthy, n), (mother, n), (ass, n)...","[fuck, filthy, mother, dry]",fuck filthy mother dry


In [12]:
# NMF
no_features = 1000

tfidf = TfidfVectorizer(max_features=no_features)
tfidf_features = tfidf.fit_transform(train_data.lemmatize_text)
tfidf_feature_names = tfidf.get_feature_names()


cv = CountVectorizer(max_features=no_features)
cv_features = cv.fit_transform(train_data.lemmatize_text)
cv_feature_names = cv.get_feature_names()

no_topics=4
nmf_tfidf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_features)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("主题 {} : {}".format(topic_idx,"|".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])))

no_top_words = 4
print('---------------NMF-tfidf_features 主题-----------------------------------------')
display_topics(nmf_tfidf, tfidf_feature_names, no_top_words)
print()

---------------NMF-tfidf_features 主题-----------------------------------------
主题 0 : will|people|go|one
主题 1 : trump|president|liar|donald
主题 2 : fuck|go|fucking|shit
主题 3 : stupid|comment|fix|people



In [14]:
mdl=tp.LDAModel(k=4,min_df=2,seed=555)
for words in train_data['lemmatize_word_w_pos']:
    if words:
        mdl.add_doc(words=words)
        
mdl.train()

for k in range(mdl.k):
    print("Top 10 words of topic #{}".format(k))
    print(mdl.get_topic_words(k,top_n=10))
    print("\n")

Top 10 words of topic #0
[('fuck', 0.021477898582816124), ('go', 0.011361111886799335), ('will', 0.010454929433763027), ('people', 0.006595762912184), ('shit', 0.006508600898087025), ('stupid', 0.0060472446493804455), ('die', 0.006015685852617025), ('fucking', 0.005386016797274351), ('one', 0.005256776697933674), ('make', 0.005095978267490864)]


Top 10 words of topic #1
[('trump', 0.009790468961000443), ('people', 0.009123208001255989), ('will', 0.008409641683101654), ('hate', 0.006837659515440464), ('go', 0.006178709212690592), ('one', 0.005753656849265099), ('think', 0.00538678141310811), ('say', 0.005224121734499931), ('make', 0.005106579512357712), ('stupid', 0.00510183023288846)]


Top 10 words of topic #2
[('nigger', 0.010911394841969013), ('suck', 0.009760710410773754), ('will', 0.00958207342773676), ('trump', 0.008201252669095993), ('gay', 0.00804353691637516), ('people', 0.00697332015261054), ('faggot', 0.00646315747871995), ('go', 0.006012539844959974), ('stupid', 0.00532373

In [15]:
# 产看话题模型信息
mdl.summary()

<Basic Info>
| LDAModel (current version: 0.12.2)
| 133600 docs, 2967109 words
| Total Vocabs: 104971, Used Vocabs: 41269
| Entropy of words: 8.11241
| Entropy of term-weighted words: 8.11241
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 10, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.83556
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 2 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 4 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 555 (random seed)
| trained in version 0.12.2
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.17999525 0

In [16]:
# 可视化
pyLDAvis.enable_notebook()

topic_term_dists=np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists=np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists/=doc_topic_dists.sum(axis=1,keepdims=True)
doc_lengths=np.array([len(doc.words) for doc in mdl.docs])
vocab=list(mdl.used_vocabs)
term_frequency=mdl.used_vocab_freq

prepared_data=pyLDAvis.prepare(topic_term_dists,doc_topic_dists,doc_lengths,vocab,term_frequency,start_index=0,sort_topics=False)

# 可视化结果存到html文件中
pyLDAvis.save_html(prepared_data,'ldavis.html')

# notebook中显示
pyLDAvis.display(prepared_data)

In [ ]:
# 创建模型
device = 'gpu:0,1' if torch.cuda.is_available() else 'cpu'

model=BERTopic(nr_topics=4)
toxic_data=pd.read_csv('train_data.csv')
toxic_data=toxic_data[toxic_data['toxic']==1]
docs=list(toxic_data.lemmatize_word_w_pos.values)
topics,probabilities=model.fit_transform(docs)


In [ ]:
model.get_topic_info()

In [ ]:
# 可视化主体间距离
visualize_topics=model.visualize_topics()

visualize_topics

In [ ]:
#聚类
model.visualize_hierarchy(top_n_topics=4)

In [ ]:
# topics词条形图
model.visualize_barchart(topic[1])

In [ ]:
# 话题相似热力图
topic_similar_heatmap=model.visualize_heatmap(n_clusters=4)
topic_silimar_heatmap